In [ ]:
import os
print(os.getcwd())
! mkdir -p prod && mkdir -p log && which python

## Context

Different file formats and usage scenarios of the dataconverter will typically use a different combination of instance data<br>
for concepts, one file format may report the voltage and microscope fabrication details, the next not etc.<br>
Therefore, possibly not all required as well as recommended concepts from an appdef will be populatable during<br>
parsing (i.e. running dataconverter) unless the missing metadata are streamed in from other sources (e.g. custom ELNs).<br>
Consequently, the validation part of the dataconverter may issue WARNINGS:: about missing or other non-compliances of<br>
an actual set of content with instance data that is written to the NeXus/HDF5 file.<br>

## What does this code do?

This code here fishes these warnings from a set of e.g. batch-processed conversion tasks performed to support developers<br>
with detecting and fixing common patterns of cases where the application definition is not fulfilled completely.<br>

In [ ]:
# inspect run_tests.ipynb to see which tests are available

In [ ]:
root_directory = f"{os.getcwd()}/log"
verbose = True
issues = {}
whitelist = []  # "stderr.nxs_nion.2022-02-18_Metadata_Kuehbach.zip.txt"]
blacklist = []
# ["stderr.nxs_nion.2022-02-18_Metadata_Kuehbach.zip.txt",
# "stderr.image_diffraction_pattern.original_data.zip.txt"]
for root, dirs, files in os.walk(root_directory):
    for file in files:
        fpath = f"{root}/{file}".replace(os.sep * 2, os.sep)
        fname = os.path.basename(fpath)
        if fname.startswith("stderr.") and fname.endswith(".txt"):
            # print(f"{fname}")
            if len(whitelist) > 0:
                if fname not in whitelist:
                    continue
            if len(blacklist) > 0:
               if fname in blacklist:
                    continue
            with open(fpath, mode="r", encoding="utf8") as fp:
                txt = fp.read()
                txt = txt.replace("\r\n", "\n")  # windows to unix EOL conversion
                tokenized = [line for line in txt.split("\n") if line.strip() != "" and line.startswith("#") is False]
                for line in tokenized:
                    if line.startswith("WARNING: "):
                        if line not in issues:
                            issues[line] = [fname]
                        else:
                            issues[line].append(fname)
                del txt
n_issues = 0
for key, val in issues.items():  # sorted()
    if verbose and (not any(token in key for token in ["is being written", "written without documentation"])):
        print(f"{key}, {len(val)}")
        #for entry in val:
        #    print(f"\t{entry[len('stderr.'):]}")
        # print(f"{key}, {len(val)}")  # {val}")
        n_issues += 1
print(f"There are {n_issues} types of WARNINGs across the set")